In [0]:
%sql

-- criando a schema gold
CREATE SCHEMA IF NOT EXISTS gold;


In [0]:
%sql 

CREATE OR REPLACE TABLE gold.dim_candidato
USING DELTA AS 
SELECT
      monotonically_increasing_id() AS SK_CANDIDATO,-- Criando uma Surrogate key
      sha2(
          concat_ws('|', 
              FAIXA_ETARIA, SEXO, COR_RACA, TIPO_ESCOLA, SG_UF_PROVA, 
              ESCOLARIDADE_PAI, ESCOLARIDADE_MAE, RENDA_FAMILIAR
          ), 256
      ) AS CANDIDATO_KEY, -- criando uma chave de negocio


      FAIXA_ETARIA, 
      SEXO, 
      COR_RACA,
      TIPO_ESCOLA, 
      SG_UF_PROVA,
      ESCOLARIDADE_PAI,
      ESCOLARIDADE_MAE,
      RENDA_FAMILIAR

FROM silver.enem2023_silver
GROUP BY 
      FAIXA_ETARIA, 
      SEXO, 
      COR_RACA,
      TIPO_ESCOLA, 
      SG_UF_PROVA,
      ESCOLARIDADE_PAI,
      ESCOLARIDADE_MAE,
      RENDA_FAMILIAR;
  


In [0]:
%sql
SELECT *
FROM gold.dim_candidato

In [0]:
%sql 

CREATE OR REPLACE TABLE gold.dim_local -- escola onde o ensino médio foi cursado
USING DELTA AS 
SELECT
      monotonically_increasing_id() AS SK_LOCAL,-- Criando uma Surrogate key
      SG_UF_PROVA
      
FROM silver.enem2023_silver

GROUP BY SG_UF_PROVA

In [0]:
%sql
SELECT *
FROM gold.dim_local

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fato_enem2023
USING DELTA AS 

WITH preparacao_fato AS (
    SELECT
        s.*,
        -- RECRIANDO A CHAVE DE NEGOCIO
        sha2(
            concat_ws('|', 
                FAIXA_ETARIA, SEXO, COR_RACA, TIPO_ESCOLA, SG_UF_PROVA, 
                ESCOLARIDADE_PAI, ESCOLARIDADE_MAE, RENDA_FAMILIAR
            ), 256
        ) AS CANDIDATO_KEY
    FROM silver.enem2023_silver s
)

SELECT
    s.NU_INSCRICAO, 

-- CHAVES ESTRANGEIRAS
    C.SK_CANDIDATO,
    l.SK_LOCAL,

-- Notas
    s.NOTA_CN,
    s.NOTA_CH,
    s.NOTA_LC,
    s.NOTA_MT,
    s.NOTA_REDACAO,

-- média das 5 disciplinas
    ROUND((NOTA_CN +
        NOTA_MT +
        NOTA_CH +
        NOTA_LC +
        NOTA_REDACAO)
     / 5,2) AS MEDIA_DISCIPLINAS,

-- Presenças
    s.PRESENCA_CN,
    s.PRESENCA_CH,
    s.PRESENCA_LC,
    s.PRESENCA_MT,
    s.TP_STATUS_REDACAO

    FROM preparacao_fato s
    LEFT JOIN gold.dim_candidato c
    ON  s.candidato_key = c.CANDIDATO_KEY
    LEFT JOIN gold.dim_local l
    ON s.SG_UF_PROVA = l.SG_UF_PROVA
    


In [0]:
%sql
SELECT * FROM gold.fato_enem2023
LIMIT 10

### CRIAÇÃO DE TABELAS GOLD AGREGADAS
As próximas etapas tem como o objetivo a criação das tabelas gold agregadas cuja a intenção é facilitar as consultas dos principais problemas de negócio relacionadas a análise dos dados do Enem e otimizar a resposta das perguntas presentes no objetivo desse MVP.

In [0]:
%sql
-- As tabelas criadas a seguir tem como objetivo analisar o perfil dos inscritos no enem

-- criação da tabela gold.perfil_inscritos_por_faixa_etaria
    CREATE OR REPLACE TABLE gold.perfil_inscritos_faixa_etaria
    SELECT 
    c.FAIXA_ETARIA, 
    COUNT(*) AS TOTAL_FAIXA_ETARIA,
    ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0 / (SELECT COUNT(*) FROM gold.fato_enem2023),2) AS PERCENTUAL_FAIXA_ETARIA

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO 
    GROUP BY
        c.FAIXA_ETARIA
    ORDER BY TOTAL_FAIXA_ETARIA DESC;

-- criação da tabela gold.perfil_inscritos_sexo
    CREATE OR REPLACE TABLE gold.perfil_inscritos_sexo
    SELECT 
        c.SEXO, 
        COUNT(*) AS TOTAL_POR_SEXO,
        ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0/ (SELECT COUNT(*) FROM gold.fato_enem2023), 2) AS PERCENTUAL_POR_SEXO
        

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO 
    GROUP BY
        c.SEXO
    ORDER BY COUNT(*) DESC;


-- criação da tabela gold.perfil_inscritos_por_raca
    CREATE OR REPLACE TABLE gold.perfil_inscritos_por_raca
    SELECT
        c.COR_RACA,
        COUNT(*) AS TOTAL_POR_RACA,
        -- Divide a contagem do grupo pelo total de linhas e multiplica por 100
        ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0 / (SELECT COUNT(*) FROM gold.fato_enem2023),2) AS PERCENTUAL_POR_RACA
    FROM
        gold.fato_enem2023 f
    LEFT JOIN
        gold.dim_candidato c ON f.SK_CANDIDATO = c.SK_CANDIDATO
    GROUP BY
        c.COR_RACA
    ORDER BY
        PERCENTUAL_POR_RACA
 DESC;

-- criação da tabela gold.perfil_inscritos_por_tp_escola
    CREATE OR REPLACE TABLE gold.perfil_inscritos_tp_escola
    SELECT 
        c.TIPO_ESCOLA, 
        COUNT(*) AS TOTAL_TIPO_ESCOLA,
        ROUND((CAST(COUNT(*) AS DECIMAL) * 100.0 / (SELECT COUNT(*) FROM gold.fato_enem2023)),2) AS PERCENTUAL_TIPO_ESCOLA

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO 
    GROUP BY
        c.TIPO_ESCOLA
    ORDER BY PERCENTUAL_TIPO_ESCOLA DESC;
-- criação da tabela gold.perfil_inscritos_por_uf
    CREATE OR REPLACE TABLE gold.perfil_inscritos_por_uf
    SELECT 
        c.SG_UF_PROVA, 
        COUNT(*) AS TOTAL_POR_UF,
        ROUND((CAST(COUNT(*) AS DECIMAL) * 100.0 / (SELECT COUNT(*) FROM gold.fato_enem2023)),2) AS PERCENTUAL_POR_UF

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO 
    GROUP BY
        c.SG_UF_PROVA
    ORDER BY PERCENTUAL_POR_UF DESC;


-- criação da tabela gold.perfil_inscritos_por_renda
    CREATE OR REPLACE TABLE gold.perfil_inscritos_renda
    SELECT 
    c.RENDA_FAMILIAR, 
    COUNT(*) AS TOTAL_RENDA_FAMILIAR,
    ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0 / (SELECT COUNT(*) FROM gold.fato_enem2023),2) AS PERCENTUAL_RENDA_FAMILIAR

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO 
    GROUP BY
        c.RENDA_FAMILIAR
    ORDER BY COUNT(*) DESC
    



In [0]:
%sql
-- As tabelas criadas a seguir tem como objetivo analisar o perfil dos candidatos que faltaram em pelo menos um dos dias do enem. Como a prova de Ciência Humanas e Ciências da Natureza são em dias diferentes, estou filtrando os candidatos que faltaram em pelo menos uma dessas provas do enem.

-- criação da tabela gold.perfil_faltantes_por_faixa_etaria
    CREATE OR REPLACE TABLE gold.perfil_faltantes_faixa_etaria
    WITH total_faltantes AS (
    SELECT COUNT(*) AS total
    FROM gold.fato_enem2023
    WHERE PRESENCA_CH = 'Faltou'
       OR PRESENCA_CN = 'Faltou'
    )   
    SELECT 
    c.FAIXA_ETARIA, 
    COUNT(*) AS TOTAL_FAIXA_ETARIA,
    ROUND(
        CAST(COUNT(*) AS DECIMAL) * 100.0 / tf.total, 2
    ) AS PERCENTUAL_FAIXA_ETARIA
    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO
    CROSS JOIN total_faltantes tf
    WHERE f.PRESENCA_CH = 'Faltou' 
    OR f.PRESENCA_CN = 'Faltou'
    GROUP BY c.FAIXA_ETARIA, tf.total
    ORDER BY PERCENTUAL_FAIXA_ETARIA DESC;

-- criação da tabela gold.perfil_faltantes_sexo
    CREATE OR REPLACE TABLE gold.perfil_faltantes_sexo
    WITH total_faltantes AS (
    SELECT COUNT(*) AS total
    FROM gold.fato_enem2023
    WHERE PRESENCA_CH = 'Faltou'
       OR PRESENCA_CN = 'Faltou'
    )
    SELECT 
        c.SEXO, 
        COUNT(*) AS TOTAL_POR_SEXO,
        ROUND(
            CAST(COUNT(*) AS DECIMAL) * 100.0 / tf.total, 
            2
        ) AS PERCENTUAL_POR_SEXO
    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO
    CROSS JOIN total_faltantes tf

    WHERE f.PRESENCA_CH = 'Faltou' 
    OR f.PRESENCA_CN = 'Faltou'
    GROUP BY c.SEXO, tf.total
    ORDER BY TOTAL_POR_SEXO DESC;


-- criação da tabela gold.perfil_faltantes_por_raça
    CREATE OR REPLACE TABLE gold.perfil_faltantes_por_raca
    WITH total_faltantes AS (
        SELECT COUNT(*) AS total
        FROM gold.fato_enem2023
        WHERE PRESENCA_CH = 'Faltou'
        OR PRESENCA_CN = 'Faltou'
    )
    SELECT
        c.COR_RACA,
        COUNT(*) AS TOTAL_POR_RACA,
        ROUND(
            CAST(COUNT(*) AS DECIMAL) * 100.0 / tf.total, 2
        ) AS PERCENTUAL_POR_RACA
    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c 
        ON f.SK_CANDIDATO = c.SK_CANDIDATO
    CROSS JOIN total_faltantes tf
    WHERE f.PRESENCA_CH = 'Faltou' 
    OR f.PRESENCA_CN = 'Faltou'
    GROUP BY c.COR_RACA, tf.total
    ORDER BY PERCENTUAL_POR_RACA DESC;

-- criação da tabela gold.perfil_faltantes_por_tp_escola
    CREATE OR REPLACE TABLE gold.perfil_faltantes_tp_escola
    WITH total_faltantes AS (
    SELECT COUNT(*) AS total
    FROM gold.fato_enem2023
    WHERE PRESENCA_CH = 'Faltou'
       OR PRESENCA_CN = 'Faltou'
    )
    SELECT 

        c.TIPO_ESCOLA, 
        COUNT(*) AS TOTAL_TIPO_ESCOLA,
        ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0 / tf.total,2) AS PERCENTUAL_TIPO_ESCOLA

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
    ON f.SK_CANDIDATO = c.SK_CANDIDATO
    CROSS JOIN total_faltantes tf
    WHERE f.PRESENCA_CH = 'Faltou' 
    OR f.PRESENCA_CN = 'Faltou'
    GROUP BY
        c.TIPO_ESCOLA, tf.total
    ORDER BY PERCENTUAL_TIPO_ESCOLA DESC;


-- criação da tabela gold.perfil_faltantes_por_uF
    CREATE OR REPLACE TABLE gold.perfil_faltantes_por_uf
    WITH total_faltantes AS (
    SELECT COUNT(*) AS total
    FROM gold.fato_enem2023
    WHERE PRESENCA_CH = 'Faltou'
       OR PRESENCA_CN = 'Faltou'
    )
    SELECT 
        c.SG_UF_PROVA, 
        COUNT(*) AS TOTAL_POR_UF,
        ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0 / tf.total,2) AS PERCENTUAL_POR_UF

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
    ON f.SK_CANDIDATO = c.SK_CANDIDATO
    CROSS JOIN total_faltantes tf
    WHERE f.PRESENCA_CH = 'Faltou' 
    OR f.PRESENCA_CN = 'Faltou'
    GROUP BY
        c.SG_UF_PROVA, tf.total
    ORDER BY PERCENTUAL_POR_UF DESC;


-- criação da tabela gold.perfil_faltantes_por_renda
    CREATE OR REPLACE TABLE gold.perfil_faltantes_renda
    WITH total_faltantes AS (
    SELECT COUNT(*) AS total
    FROM gold.fato_enem2023
    WHERE PRESENCA_CH = 'Faltou'
       OR PRESENCA_CN = 'Faltou'
    )
    SELECT 
    c.RENDA_FAMILIAR, 
    COUNT(*) AS TOTAL_RENDA_FAMILIAR,
    ROUND(CAST(COUNT(*) AS DECIMAL) * 100.0 / tf.total,2) AS PERCENTUAL_RENDA_FAMILIAR

    FROM gold.fato_enem2023 f
    LEFT JOIN gold.dim_candidato c
        ON f.SK_CANDIDATO = c.SK_CANDIDATO
    CROSS JOIN total_faltantes tf
    WHERE f.PRESENCA_CH = 'Faltou' 
    OR f.PRESENCA_CN = 'Faltou'
    GROUP BY
        c.RENDA_FAMILIAR, tf.total
    ORDER BY COUNT(*) DESC

In [0]:
%sql
CREATE OR REPLACE TABLE gold.media_por_uf 
SELECT 
ROUND(AVG(f.NOTA_CN),2) AS MEDIA_CN,
ROUND(AVG(f.NOTA_MT),2) AS MEDIA_MT,
ROUND(AVG(f.NOTA_CH),2) AS MEDIA_CH,
ROUND(AVG(f.NOTA_LC),2) AS MEDIA_LC,
ROUND(AVG(f.NOTA_REDACAO),2) AS MEDIA_REDACAO,

 -- média das 5 disciplinas
    ROUND( (
        AVG(f.NOTA_CN) +
        AVG(f.NOTA_MT) +
        AVG(f.NOTA_CH) +
        AVG(f.NOTA_LC) +
        AVG(f.NOTA_REDACAO)
    ) / 5,2) AS MEDIA_DISCIPLINAS,

SG_UF_PROVA AS UF
FROM workspace.gold.fato_enem2023 f 
LEFT JOIN workspace.gold.dim_local l
ON f.SK_LOCAL = l.SK_LOCAL 
WHERE 
    f.PRESENCA_CH = 'Presente' AND 
    f.PRESENCA_CN = 'Presente' AND 
    TP_STATUS_REDACAO = 'Sem problemas'   
    --considerando apenas os candidatos que fizeram as provas nos dois dias de prova e tiveram notas válidas na redação

GROUP BY SG_UF_PROVA
ORDER BY MEDIA_DISCIPLINAS DESC

In [0]:
%sql
CREATE OR REPLACE TABLE gold.notas_validas
SELECT 
NU_INSCRICAO,
SK_CANDIDATO,
SK_LOCAL,
NOTA_CN,
NOTA_CH,
NOTA_LC,
NOTA_MT,
NOTA_REDACAO,
MEDIA_DISCIPLINAS
FROM gold.fato_enem2023
WHERE PRESENCA_CH = 'Presente' AND PRESENCA_CN = 'Presente'
AND TP_STATUS_REDACAO = 'Sem problemas'